# The Battle of Neighborhoods
## Coursera - Applied Data Science Capstone Project

### Project Description
I love pizza!  I like deep dish, New York style, thin crust and even Pizza rolls.  COVID-19 has changed by work life and I can now move to any city I want as long as I have a good internet connection.  I want to find a new medium sized city with the most Pizza places per capita.  I will then try to find a spot in that city to look for my new home.

### Description of the Data
#### Where is the Data Located?
The Data for my project is located in two sources.  The first is Wikipedia for the citys in the US. It contains a list of the largest 300 cities in the use.  I will use FourSquare to find the number and location of the pizza restaurants around each city.

#### How will the data be used to solve the problem?
The data will be used to first find a list of medium cities.  I will filter that long list of citys to a subset of medium sized cities (250K < Pop <1M).  After I have a shorter list of cities, I will then look for the number of pizza places within 5 km of city center.  I will then find the per Capita number of Pizza locations per city.  I will then select the city with the most Pizza restaraunts per capita and plot the locations.  After I have a general idea where they are located I will cluster the list of Pizza shops into groups.  After I have clustered the locations, I will find the group with the highest number of locations in the cluster and start looking for a house around that centroid.

## Step 1. Scrape Wikipedia for the list of Neighborhoods

In [2]:
# Import Key Libraries into the Notebook
import pandas as pd
import numpy as np

In [3]:
import requests # library to handle requests
import random # library for random number generation


In [4]:
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [6]:
# module to convert an address into latitude and longitude values
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
print('GeoCoders installed')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0          conda-forge
    geopy:         2.0.0-pyh9f0ad1d_0 conda-forge


geopy-2.0.0          | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
GeoCoders installed


In [7]:
# import the library we use to open URLs
import urllib.request as URLRequest

### Now that all the libraries are Imported, We can move on to working with data

### Read in the web address of US Cities by Population

In [8]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"

In [9]:
# open the url using urllib.request and put the HTML into the page variable
page = URLRequest.urlopen(url)

In [10]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [11]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [12]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of United States cities by population - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"04059fbf-36f9-4119-8245-e104660d26c5","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_United_States_cities_by_population","wgTitle":"List of United States cities by population","wgCurRevisionId":961764896,"wgRevisionId":961764896,"wgArticleId":1649321,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Wikipedia indefinitely semi-pr

In [13]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables=soup.find_all("table")
all_tables

[<table class="noprint infobox" id="GeoGroup" style="width: 23em; font-size: 88%; line-height: 1.5em">
 <tbody><tr>
 <td><b>Map all coordinates using:</b> <a class="external text" href="//tools.wmflabs.org/osm4wiki/cgi-bin/wiki/wiki-osm.pl?project=en&amp;article=List_of_United_States_cities_by_population">OpenStreetMap</a> 
 </td></tr>
 <tr>
 <td><b>Download coordinates as:</b> <a class="external text" href="//tools.wmflabs.org/kmlexport?article=List_of_United_States_cities_by_population">KML</a> <b>·</b> <a class="external text" href="http://tripgang.com/kml2gpx/http%3A%2F%2Ftools.wmflabs.org%2Fkmlexport%3Farticle%3DList_of_United_States_cities_by_population?gpx=1" rel="nofollow">GPX</a>
 </td></tr></tbody></table>,
 <table class="vertical-navbox nowraplinks navbox" style="float:right;clear:right;width:22.0em;margin:0 0 1.0em 1.0em;background:#f8f9fa;border:1px solid #aaa;padding:0.2em;border-spacing:0.4em 0;text-align:center;line-height:1.4em;font-size:88%"><tbody><tr><th style="padd

In [14]:
right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable" style="text-align:center">
<tbody><tr>
<th>2019<br/>rank
</th>
<th>City
</th>
<th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[c]</a></sup>
</th>
<th>2019<br/>estimate
</th>
<th>2010<br/>Census
</th>
<th>Change
</th>
<th colspan="2">2016 land area
</th>
<th colspan="2">2016 population density
</th>
<th>Location
</th></tr>
<tr>
<td>1
</td>
<td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York City">New York</a></i><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[d]</a></sup>
</td>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="1137" data-file-width="2276" decoding="async" height="11" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/23px-Flag_of_New_York.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/35px-Flag_of_New_York.svg.png 1.5x, //upload.wikimedia.org

### Read the HTML into a dataset and add to the Pandas Datafrome

In [15]:
# The data is spread across 11 columns so grab all data and keep what is important
A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
I=[]
J=[]
K=[]
L=[]
for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==11:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        H.append(cells[6].find(text=True))
        I.append(cells[7].find(text=True))
        J.append(cells[8].find(text=True))
        K.append(cells[9].find(text=True))
        L.append(cells[10].find(text=True))
        
        
                

### This is a full list of potential cities

In [16]:

# Create the Dataframe and assign the array for each field to a column
df=pd.DataFrame(A,columns=['Number'])
df['CityName']=B
df['Population']=D
df

,Number,CityName,Population
0,1,New York,"8,336,817"
1,2,Los Angeles,"3,979,576"
2,3,Chicago,"2,693,976"
3,4,Houston,"2,320,268"
4,5,Phoenix,"1,680,992"
5,6,Philadelphia,"1,584,064"
6,7,San Antonio,"1,547,253"
7,8,San Diego,"1,423,851"
8,9,Dallas,"1,343,573"
9,10,San Jose,"1,021,795"


In [17]:
# Check the data types before I filter the cities for size
df.dtypes


Number        object
CityName      object
Population    object
dtype: object

## Step 2. Filter the list down to medium sized cities

### Now that I have a list of US cities, I need to convert from object to a number 

In [18]:
# Remove the commas from the numbers, and remove the '\n' from the end of every number
df['Population']=df['Population'].str.replace(',','')
df['Population']=df['Population'].str.replace('\n','')

In [19]:
# Convert the data type from Object to Float
df["Population"] = df['Population'].astype(float)

### Filter out Cities that are too small or too large

In [20]:
# Remove all cities with a population less than 250,000
df1 = df.loc[df['Population'] > 250000]
df1

,Number,CityName,Population
0,1,New York,8336817.0
1,2,Los Angeles,3979576.0
2,3,Chicago,2693976.0
3,4,Houston,2320268.0
4,5,Phoenix,1680992.0
5,6,Philadelphia,1584064.0
6,7,San Antonio,1547253.0
7,8,San Diego,1423851.0
8,9,Dallas,1343573.0
9,10,San Jose,1021795.0


In [21]:
# Remove the cities with a population of greater than 1 million from the list
dfMediumCity = df1.loc[df['Population'] < 1000000]
dfMediumCity

,Number,CityName,Population
10,11,Austin,978908.0
11,12,Jacksonville,911507.0
12,13,Fort Worth,909585.0
13,14,Columbus,898553.0
14,15,Charlotte,885708.0
15,16,San Francisco,881549.0
16,17,Indianapolis,876384.0
17,18,Seattle,753675.0
18,19,Denver,727211.0
19,20,Washington,705749.0


### Now that I have a list of potential medium sized cities, I will look up their Coordinates for each city

In [22]:
# Loop through the rows in the dataframe and request the location.
Lat=[]
Long=[]

for index, row in dfMediumCity.iterrows():
    address= row['CityName']
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    # Break the Location data into Latitude and Longitude
    latitude = location.latitude
    longitude = location.longitude
    Lat.append(latitude)
    Long.append(longitude)

In [23]:
# Update the Dataframe with the Latitude and Longitude of every city
dfMediumCity['Latitude']=Lat
dfMediumCity['Longitude']=Long
dfMediumCity

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Number,CityName,Population,Latitude,Longitude
10,11,Austin,978908.0,30.271129,-97.743700
11,12,Jacksonville,911507.0,30.332184,-81.655651
12,13,Fort Worth,909585.0,32.753177,-97.332746
13,14,Columbus,898553.0,39.962260,-83.000707
14,15,Charlotte,885708.0,35.227200,-80.843083
15,16,San Francisco,881549.0,37.779026,-122.419906
16,17,Indianapolis,876384.0,39.768333,-86.158350
17,18,Seattle,753675.0,47.603832,-122.330062
18,19,Denver,727211.0,39.739236,-104.984862
19,20,Washington,705749.0,38.894985,-77.036571


## Step 3. Find the number of Pizza Locations in a 5km circle around the city

### Now that I have a great list of potential cities, I need to find out which ones have Pizza

In [24]:
# The code was removed by Watson Studio for sharing.

In [25]:
latitude = '38.254238'
longitude = '-85.759407'

In [26]:
search_query = 'Pizza'
# Radius of 5 km around city center
radius = 50000

In [27]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VCDOEWVG3P3ALVKGAOQIC5MPMCQ01FT5HY35MK1B01SW4TCC&client_secret=IJSCBFL1KMKV3PWK5W4O0JJ0OKNFGRL0HSA22T5LW0UVFVJ4&ll=38.254238,-85.759407&v=20180604&query=Pizza&radius=50000&limit=30'

In [28]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efb0803564b600a12d4578c'},
 'response': {'venues': [{'id': '4b5b7066f964a52038fe28e3',
    'name': 'Sicilian Pizza & Pasta',
    'location': {'address': '631 S 4th St',
     'lat': 38.24796985389662,
     'lng': -85.75802815295759,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.24796985389662,
       'lng': -85.75802815295759},
      {'label': 'entrance', 'lat': 38.247937, 'lng': -85.75796}],
     'distance': 708,
     'postalCode': '40202',
     'cc': 'US',
     'city': 'Louisville',
     'state': 'KY',
     'country': 'United States',
     'formattedAddress': ['631 S 4th St',
      'Louisville, KY 40202',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '178816

In [29]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",1788162,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/sicilian-pi...,False,4b5b7066f964a52038fe28e3,631 S 4th St,...,NaN,708,"[631 S 4th St, Louisville, KY 40202, United St...","[{'label': 'display', 'lat': 38.24796985389662...",38.247970,-85.758028,40202,KY,Sicilian Pizza & Pasta,v-1593509670
1,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c8edaebd68c6dcbb4eeffa1,110 W Main St,...,2nd St,646,"[110 W Main St (2nd St), Louisville, KY 40202,...","[{'label': 'display', 'lat': 38.25605521414837...",38.256055,-85.752387,40202,KY,Impellizzeri's Pizza,v-1593509670
2,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,5b9aa75673fe25002c39fafc,445 S 4th St,...,NaN,310,"[445 S 4th St, Louisville, KY 40202, United St...","[{'label': 'display', 'lat': 38.252133, 'lng':...",38.252133,-85.757084,40202,KY,Pizza Bar,v-1593509670
3,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3272b219836c91c7d8cac6,801 W Broadway,...,NaN,951,"[801 W Broadway, Louisville, KY 40202, United ...","[{'label': 'display', 'lat': 38.2471809387207,...",38.247181,-85.765556,40202,KY,Pizza Hut,v-1593509670
4,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",NaN,NaN,NaN,NaN,NaN,NaN,False,56cf4080498e5d8a7abb95d1,NaN,...,NaN,334,"[Louisville, KY, United States]","[{'label': 'display', 'lat': 38.25169461774938...",38.251695,-85.757372,NaN,KY,Birracibo Artisan Pizza,v-1593509670


In [30]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Sicilian Pizza & Pasta,Pizza Place,631 S 4th St,US,Louisville,United States,NaN,708,"[631 S 4th St, Louisville, KY 40202, United St...","[{'label': 'display', 'lat': 38.24796985389662...",38.247970,-85.758028,40202,KY,4b5b7066f964a52038fe28e3
1,Impellizzeri's Pizza,Pizza Place,110 W Main St,US,Louisville,United States,2nd St,646,"[110 W Main St (2nd St), Louisville, KY 40202,...","[{'label': 'display', 'lat': 38.25605521414837...",38.256055,-85.752387,40202,KY,4c8edaebd68c6dcbb4eeffa1
2,Pizza Bar,Pizza Place,445 S 4th St,US,Louisville,United States,NaN,310,"[445 S 4th St, Louisville, KY 40202, United St...","[{'label': 'display', 'lat': 38.252133, 'lng':...",38.252133,-85.757084,40202,KY,5b9aa75673fe25002c39fafc
3,Pizza Hut,Pizza Place,801 W Broadway,US,Louisville,United States,NaN,951,"[801 W Broadway, Louisville, KY 40202, United ...","[{'label': 'display', 'lat': 38.2471809387207,...",38.247181,-85.765556,40202,KY,4f3272b219836c91c7d8cac6
4,Birracibo Artisan Pizza,Italian Restaurant,NaN,US,Louisville,United States,NaN,334,"[Louisville, KY, United States]","[{'label': 'display', 'lat': 38.25169461774938...",38.251695,-85.757372,NaN,KY,56cf4080498e5d8a7abb95d1
5,Wick's Pizza Parlor & Pub,Pizza Place,975 Baxter Ave,US,Louisville,United States,NaN,3428,"[975 Baxter Ave, Louisville, KY 40204, United ...","[{'label': 'display', 'lat': 38.24048237015347...",38.240482,-85.724318,40204,KY,4bc2b5da4cdfc9b68f7d9621
6,Papa John's Pizza,Pizza Place,117 W Liberty St,US,Louisville,United States,NaN,579,"[117 W Liberty St, Louisville, KY 40202, Unite...","[{'label': 'entrance', 'lat': 38.252784, 'lng'...",38.252982,-85.752975,40202,KY,4cf05c8d899c6ea8ae3aeac1
7,Pizza Hut,Pizza Place,2508 Preston Hwy,US,Louisville,United States,Clark's Lane,4929,"[2508 Preston Hwy (Clark's Lane), Louisville, ...","[{'label': 'display', 'lat': 38.2122785213486,...",38.212279,-85.741380,40217,KY,4b4bc393f964a520dea626e3
8,Pizza Today Magazine,Coworking Space,NaN,US,Louisville,United States,NaN,1421,"[Louisville, KY 40203, United States]","[{'label': 'display', 'lat': 38.242734, 'lng':...",38.242734,-85.766467,40203,KY,4cc5d3b23d7fa1cd6c08b45f
9,Papa John's Pizza,Pizza Place,401 E Main St,US,Louisville,United States,NaN,1288,"[401 E Main St, Louisville, KY 40202, United S...","[{'label': 'display', 'lat': 38.25570297241211...",38.255703,-85.744789,40202,KY,4e4d518ebd413c4cc6702492


### Now that I know that I can pull the Data for one city, I need to pull a full list for every city

In [31]:
dfPizza=pd.DataFrame(columns=list(dataframe_filtered.columns))
dfPizza

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id


In [49]:
LIMIT = 500
fullresults=''

for index, row in dfMediumCity.iterrows():
    latitude= row['Latitude']
    longitude= row['Longitude']
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    results = requests.get(url).json()
    # assign relevant part of JSON to venues
    venues = results['response']['venues']
# tranform venues into a dataframe
    dataframe = json_normalize(venues)
    dfPizza=pd.concat([dfPizza, dataframe], ignore_index=True)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [50]:
dfPizza.shape

(6749, 37)

In [94]:
dfPizza.head()

,address,categories,cc,city,country,crossStreet,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,...,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,postalCode,referralId,state,venuePage.id
0,NaN,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.267108,-97.739088,NaN,78701,TX,Jack Pizza Company,NaN,v-1593509670,NaN,NaN
1,NaN,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.269797,-97.749849,NaN,78701,TX,Gatti's Pizza,NaN,v-1593509670,NaN,66438817
2,NaN,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,1577785,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",...,30.277757,-97.744391,NaN,78701,TX,iFratelli Pizza,NaN,v-1593509670,NaN,NaN
3,NaN,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.256824,-97.739480,NaN,78701,TX,Salvation Pizza,NaN,v-1593509670,NaN,148383145
4,NaN,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,30.267732,-97.740302,NaN,78701,TX,Pizzaria Papparazzi,NaN,v-1593509670,NaN,NaN


In [95]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dfPizza_filtered = dfPizza.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dfPizza_filtered['categories'] = dfPizza_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dfPizza_filtered.columns = [column.split('.')[-1] for column in dfPizza_filtered.columns]

dfPizza_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Jack Pizza Company,Pizza Place,402 E 6th St,US,Austin,United States,at Trinity St,630.0,"[402 E 6th St (at Trinity St), Austin, TX 7870...","[{'label': 'display', 'lat': 30.26710814774817...",30.267108,-97.739088,78701,TX,4b9af532f964a520dae635e3
1,Gatti's Pizza,Pizza Place,701 W 6th St,US,Austin,United States,Rio Grande St,609.0,"[701 W 6th St (Rio Grande St), Austin, TX 7870...","[{'label': 'display', 'lat': 30.26979694804358...",30.269797,-97.749849,78701,TX,4ab1609ff964a520386920e3
2,iFratelli Pizza,Pizza Place,NaN,US,Austin,United States,NaN,740.0,"[Austin, TX 78701, United States]","[{'label': 'display', 'lat': 30.277757, 'lng':...",30.277757,-97.744391,78701,TX,5c677d620719020025fb7930
3,Salvation Pizza,Pizza Place,51 Rainey St Ste 130,US,Austin,United States,NaN,1643.0,"[51 Rainey St Ste 130, Austin, TX 78701, Unite...","[{'label': 'display', 'lat': 30.25682381638263...",30.256824,-97.739480,78701,TX,54ed115c498ee35c8f7f47ff
4,Pizzaria Papparazzi,Pizza Place,607 San Jacinto Blvd,US,Austin,United States,at E 6th St,499.0,"[607 San Jacinto Blvd (at E 6th St), Austin, T...","[{'label': 'display', 'lat': 30.26773172897584...",30.267732,-97.740302,78701,TX,4b99b2fef964a520cf8c35e3
5,Pie95 Pizza,Pizza Place,Cruising Around Jacksonville,US,Jacksonville,United States,NaN,0.0,"[Cruising Around Jacksonville, Jacksonville, F...","[{'label': 'display', 'lat': 30.332188, 'lng':...",30.332188,-81.655650,32202,FL,58b61f8c4e31c52f8800b034
6,Chicago Pizza and Sports Grille,Pizza Place,2 Independent Drive,US,Jacksonville,United States,NaN,974.0,"[2 Independent Drive, Jacksonville, FL 32202, ...","[{'label': 'display', 'lat': 30.32465038030418...",30.324650,-81.660815,32202,FL,4c3ba0ad62f1c92858d88f53
7,Paddys Pizza,Pizza Place,1402 San Marco Blvd,US,Jacksonville,United States,NaN,2302.0,"[1402 San Marco Blvd, Jacksonville, FL 32207, ...","[{'label': 'display', 'lat': 30.31198692321777...",30.311987,-81.660812,32207,FL,4bab9a08f964a520feb73ae3
8,Biggie’s Pizza Riverside,Pizza Place,1053 Park St,US,Jacksonville,United States,NaN,3146.0,"[1053 Park St, Jacksonville, FL 32204, United ...","[{'label': 'display', 'lat': 30.314184, 'lng':...",30.314184,-81.680900,32204,FL,5dfadc15ba3a8b0007d8bdb7
9,Royal Pizza Plus,Italian Restaurant,"Fort Caroline, Jacksonville, FL 32277",US,Jacksonville,United States,NaN,451.0,"[Fort Caroline, Jacksonville, FL 32277, Jackso...","[{'label': 'display', 'lat': 30.33323023, 'lng...",30.333230,-81.660193,32277,FL,4eb1d28a9adfb95b77834202


In [96]:
dfPizza_filtered.groupby('city').count()

,name,categories,address,cc,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
city,,,,,,,,,,,,,,
AZ,2,0,2,2,2,2,2,2,2,2,2,2,2,2
Alameda,8,8,8,8,8,2,8,8,8,8,8,6,8,8
Albany,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Albuquerque,103,103,92,103,103,26,103,103,103,103,103,98,103,103
Alcorcón,1,1,1,1,1,0,1,1,1,1,1,1,1,1
Alexandria,7,7,7,7,7,0,7,7,7,7,7,7,7,7
Alhambra,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Allen,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Altamonte Springs,1,1,1,1,1,0,1,1,1,1,1,1,1,1


In [97]:
dfPizza_Count = dfPizza_filtered.groupby('city').count()
dfPizza_Count

,name,categories,address,cc,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
city,,,,,,,,,,,,,,
AZ,2,0,2,2,2,2,2,2,2,2,2,2,2,2
Alameda,8,8,8,8,8,2,8,8,8,8,8,6,8,8
Albany,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Albuquerque,103,103,92,103,103,26,103,103,103,103,103,98,103,103
Alcorcón,1,1,1,1,1,0,1,1,1,1,1,1,1,1
Alexandria,7,7,7,7,7,0,7,7,7,7,7,7,7,7
Alhambra,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Allen,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Altamonte Springs,1,1,1,1,1,0,1,1,1,1,1,1,1,1


In [98]:
dfPizza_Count.apply(pd.value_counts)
dfPizza_Count

,name,categories,address,cc,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
city,,,,,,,,,,,,,,
AZ,2,0,2,2,2,2,2,2,2,2,2,2,2,2
Alameda,8,8,8,8,8,2,8,8,8,8,8,6,8,8
Albany,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Albuquerque,103,103,92,103,103,26,103,103,103,103,103,98,103,103
Alcorcón,1,1,1,1,1,0,1,1,1,1,1,1,1,1
Alexandria,7,7,7,7,7,0,7,7,7,7,7,7,7,7
Alhambra,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Allen,3,3,3,3,3,2,3,3,3,3,3,3,3,3
Altamonte Springs,1,1,1,1,1,0,1,1,1,1,1,1,1,1


In [99]:
dfPizza_Count =dfPizza_filtered.groupby('city')["id"].count().reset_index(name="count")
dfPizza_Count


,city,count
0,AZ,2
1,Alameda,8
2,Albany,4
3,Albuquerque,103
4,Alcorcón,1
5,Alexandria,7
6,Alhambra,3
7,Allen,3
8,Altamonte Springs,1
9,Amherst,3


In [100]:
dfMediumCityCount = dfMediumCity.join(dfPizza_Count.set_index('city'), on='CityName')
dfMediumCityCount

,Number,CityName,Population,Latitude,Longitude,count
10,11,Austin,978908.0,30.271129,-97.743700,99.0
11,12,Jacksonville,911507.0,30.332184,-81.655651,105.0
12,13,Fort Worth,909585.0,32.753177,-97.332746,69.0
13,14,Columbus,898553.0,39.962260,-83.000707,84.0
14,15,Charlotte,885708.0,35.227200,-80.843083,103.0
15,16,San Francisco,881549.0,37.779026,-122.419906,141.0
16,17,Indianapolis,876384.0,39.768333,-86.158350,97.0
17,18,Seattle,753675.0,47.603832,-122.330062,103.0
18,19,Denver,727211.0,39.739236,-104.984862,95.0
19,20,Washington,705749.0,38.894985,-77.036571,77.0


## Step 4. Calculate the Per Capita Pizza Spots per city

In [101]:
dfMediumCityCount['perCapita'] = dfMediumCityCount['count']/dfMediumCityCount['Population']
dfMediumCityCount

,Number,CityName,Population,Latitude,Longitude,count,perCapita
10,11,Austin,978908.0,30.271129,-97.743700,99.0,0.000101
11,12,Jacksonville,911507.0,30.332184,-81.655651,105.0,0.000115
12,13,Fort Worth,909585.0,32.753177,-97.332746,69.0,0.000076
13,14,Columbus,898553.0,39.962260,-83.000707,84.0,0.000093
14,15,Charlotte,885708.0,35.227200,-80.843083,103.0,0.000116
15,16,San Francisco,881549.0,37.779026,-122.419906,141.0,0.000160
16,17,Indianapolis,876384.0,39.768333,-86.158350,97.0,0.000111
17,18,Seattle,753675.0,47.603832,-122.330062,103.0,0.000137
18,19,Denver,727211.0,39.739236,-104.984862,95.0,0.000131
19,20,Washington,705749.0,38.894985,-77.036571,77.0,0.000109


In [102]:
dfMediumCityCount.sort_values(by=['perCapita'],ascending=False)


,Number,CityName,Population,Latitude,Longitude,count,perCapita
59,60,Lexington,323152.0,38.046407,-84.497039,105.0,0.000325
58,59,Corpus Christi,326586.0,27.747725,-97.401413,101.0,0.000309
55,56,Honolulu,345064.0,21.304547,-157.855676,91.0,0.000264
51,52,Bakersfield,384145.0,35.373871,-119.019464,101.0,0.000263
49,50,New Orleans,390144.0,29.949932,-90.070116,97.0,0.000249
50,51,Wichita,389938.0,37.692236,-97.337545,95.0,0.000244
46,47,Tulsa,401190.0,36.155681,-95.992911,97.0,0.000242
45,46,Minneapolis,429606.0,44.977300,-93.265469,101.0,0.000235
43,44,Virginia Beach,449974.0,36.852984,-75.977418,103.0,0.000229
80,81,Chandler,261165.0,33.306222,-111.841332,59.0,0.000226


## Based on the perCapita Column, I am moving to Lexington!

## Step 5. Cluster the Pizza locations in to 5 groups to find the highest density location

### Now lets see if I can find a neighborhood to move to

In [103]:
dfPizzaLex= dfPizza_filtered[dfPizza_filtered['city'] == 'Lexington']
dfPizzaLex


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
242,La Gourmet Pizza,Pizza Place,325 W Main St,US,Lexington,United States,NaN,374.0,"[325 W Main St, Lexington, KY 40507, United St...","[{'label': 'display', 'lat': 38.04895001941646...",38.048950,-84.499832,40507,KY,4c02d07c187ec9285ccab47b
243,west coast Gourmet Pizza,Pizza Place,325 W Main St #140,US,Lexington,United States,NaN,314.0,"[325 W Main St #140, Lexington, KY 40507, Unit...","[{'label': 'display', 'lat': 38.048645, 'lng':...",38.048645,-84.499229,40507,KY,51730305498ece19a66bdca0
244,Pie Five Pizza,Pizza Place,"867 S Broadway, Ste 130",US,Lexington,United States,NaN,1727.0,"[867 S Broadway, Ste 130, Lexington, KY 40504,...","[{'label': 'display', 'lat': 38.03973, 'lng': ...",38.039730,-84.514829,40504,KY,574341eb498e70237a6738b9
245,Pizza Magia,Pizza Place,319 S Limestone,US,Lexington,United States,NaN,573.0,"[319 S Limestone, Lexington, KY 40508, United ...","[{'label': 'entrance', 'lat': 38.043043, 'lng'...",38.043167,-84.502121,40508,KY,4f32569119836c91c7cde760
246,Puccini's Pizza & Pasta,Pizza Place,833 Chevy Chase Pl,US,Lexington,United States,NaN,2046.0,"[833 Chevy Chase Pl, Lexington, KY 40502, Unit...","[{'label': 'display', 'lat': 38.02868022157393...",38.028680,-84.490857,40502,KY,4b928c57f964a5202e0334e3
2793,La Gourmet Pizza,Pizza Place,325 W Main St,US,Lexington,United States,NaN,374.0,"[325 W Main St, Lexington, KY 40507, United St...","[{'label': 'display', 'lat': 38.04895001941646...",38.048950,-84.499832,40507,KY,4c02d07c187ec9285ccab47b
2794,west coast Gourmet Pizza,Pizza Place,325 W Main St #140,US,Lexington,United States,NaN,314.0,"[325 W Main St #140, Lexington, KY 40507, Unit...","[{'label': 'display', 'lat': 38.048645, 'lng':...",38.048645,-84.499229,40507,KY,51730305498ece19a66bdca0
2795,Pie Five Pizza,Pizza Place,"867 S Broadway, Ste 130",US,Lexington,United States,NaN,1727.0,"[867 S Broadway, Ste 130, Lexington, KY 40504,...","[{'label': 'display', 'lat': 38.03973, 'lng': ...",38.039730,-84.514829,40504,KY,574341eb498e70237a6738b9
2796,Pizza Magia,Pizza Place,319 S Limestone,US,Lexington,United States,NaN,573.0,"[319 S Limestone, Lexington, KY 40508, United ...","[{'label': 'entrance', 'lat': 38.043043, 'lng'...",38.043167,-84.502121,40508,KY,4f32569119836c91c7cde760
2797,Puccini's Pizza & Pasta,Pizza Place,833 Chevy Chase Pl,US,Lexington,United States,NaN,2046.0,"[833 Chevy Chase Pl, Lexington, KY 40502, Unit...","[{'label': 'display', 'lat': 38.02868022157393...",38.028680,-84.490857,40502,KY,4b928c57f964a5202e0334e3


### I now have my filtered list of Pizza shops in Lexington.  I will now plot them to see them on a map.

In [104]:
address = 'Lexington, KY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Lexington are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Lexington are 38.0464066, -84.4970393.


In [105]:
# create map of New York using latitude and longitude values
map_Lex = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, name, address in zip(dfPizzaLex['lat'], dfPizzaLex['lng'], dfPizzaLex['name'], dfPizzaLex['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Lex)  
    
map_Lex

### Cluster Pizza Locations

In [106]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [107]:
# set number of clusters
kclusters = 5

dfPizzaLex_clustering = dfPizzaLex.drop(['name', 'categories', 'address','cc','city','country','crossStreet','distance','formattedAddress','labeledLatLngs','postalCode','state','id'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfPizzaLex_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [108]:
# add clustering labels
dfPizzaLex.insert(0, 'Cluster Labels', kmeans.labels_)

dfPizzaLex.head() # check the last columns!

,Cluster Labels,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
242,1,La Gourmet Pizza,Pizza Place,325 W Main St,US,Lexington,United States,NaN,374.0,"[325 W Main St, Lexington, KY 40507, United St...","[{'label': 'display', 'lat': 38.04895001941646...",38.048950,-84.499832,40507,KY,4c02d07c187ec9285ccab47b
243,1,west coast Gourmet Pizza,Pizza Place,325 W Main St #140,US,Lexington,United States,NaN,314.0,"[325 W Main St #140, Lexington, KY 40507, Unit...","[{'label': 'display', 'lat': 38.048645, 'lng':...",38.048645,-84.499229,40507,KY,51730305498ece19a66bdca0
244,1,Pie Five Pizza,Pizza Place,"867 S Broadway, Ste 130",US,Lexington,United States,NaN,1727.0,"[867 S Broadway, Ste 130, Lexington, KY 40504,...","[{'label': 'display', 'lat': 38.03973, 'lng': ...",38.039730,-84.514829,40504,KY,574341eb498e70237a6738b9
245,1,Pizza Magia,Pizza Place,319 S Limestone,US,Lexington,United States,NaN,573.0,"[319 S Limestone, Lexington, KY 40508, United ...","[{'label': 'entrance', 'lat': 38.043043, 'lng'...",38.043167,-84.502121,40508,KY,4f32569119836c91c7cde760
246,1,Puccini's Pizza & Pasta,Pizza Place,833 Chevy Chase Pl,US,Lexington,United States,NaN,2046.0,"[833 Chevy Chase Pl, Lexington, KY 40502, Unit...","[{'label': 'display', 'lat': 38.02868022157393...",38.028680,-84.490857,40502,KY,4b928c57f964a5202e0334e3


In [109]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dfPizzaLex['lat'], dfPizzaLex['lng'], dfPizzaLex['name'], dfPizzaLex['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [111]:
dfPizzaLex.groupby('Cluster Labels').count()


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
Cluster Labels,,,,,,,,,,,,,,,
0,20,20,20,20,20,20,0,20,20,20,20,20,20,20,20
1,59,59,59,59,59,59,8,59,59,59,59,59,57,59,59
2,10,10,10,10,10,10,4,10,10,10,10,10,10,10,10
3,8,8,8,8,8,8,2,8,8,8,8,8,8,8,8
4,8,8,8,8,8,8,0,8,8,8,8,8,8,8,8


In [112]:
#Find the Center of the Clusters
k_means_cluster_centers = kmeans.cluster_centers_
k_means_cluster_centers

array([[ 38.05804447, -84.4676803 ],
       [ 38.04341597, -84.50643466],
       [ 37.99456138, -84.5439353 ],
       [ 37.97353283, -84.48902198],
       [ 38.01083955, -84.43551451]])

## The Cluster with the Most Pizza Locations is 1 with 59

## The center of Cluster 1 is 38.04341597, -84.50643466 so I will look for a new home here!!!!